# New  for O$_3$ analysis over Japan. Wind
## import

In [1]:
import calendar
import netCDF4
import matplotlib.pyplot as plt
import os
os.environ["PROJ_LIB"] = "C:/Users/admin/Anaconda3/Library/share"
from mpl_toolkits.basemap import Basemap
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import interpolate

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Init

In [2]:
# ----------- ACTM params
# --- Define lons
act_grid = 2.8125
act_lons = np.arange(0, 360, act_grid)
act_nlon = len(act_lons)

# --- Define lats
act_lats = [87.8638, 85.09653, 82.31291, 79.52560, 76.73690, 73.94752, 71.15775, 68.36776,
            65.57761, 62.78735, 59.99702, 57.20663, 54.4162, 51.62573, 48.83524, 46.04473,
            43.25420, 40.46365, 37.67309, 34.88252, 32.09195, 29.30136, 26.51077, 23.72017,
            20.92957, 18.13897, 15.34836, 12.55776, 9.767145, 6.976533, 4.185921, 1.395307,
            -1.395307, -4.185921, -6.976533, -9.767145, -12.55776, -15.34836, -18.13897, -20.92957,
            -23.72017, -26.51077, -29.30136, -32.09195, -34.88252, -37.67309, -40.46365, -43.2542,
            -46.04473, -48.83524, -51.62573, -54.4162, -57.20663, -59.99702, -62.78735, -65.57761,
            -68.36776, -71.15775, -73.94752, -76.7369, -79.5256, -82.31291, -85.09653, -87.8638]
act_lats = np.array(act_lats)
act_nlat = len(act_lats)

# --- domain & data
map_lims = ['Japan', 20, 50, 115, 155, 10, 10, 60, 9, 10, 10]
wnd_dir = 'D:/OneDrive - chiba-u.jp/CH4_proj/medium/wind/'

# --- plot
path = '../'
plt_dir = path + 'plots/'

# --- month
months = calendar.month_abbr[:]
del months[0]

abc = 'abcdefghijklmnopqrstuvwxyz'

## Read wind from nc

In [3]:
def read_wind():

    def ncdump(nc_file, nc_fid, verb=True):

        # --- print atrib
        def print_ncattr(key):
            try:
                print(f'\t\t\tType: {repr(nc_fid.variables[key].dtype)}')
                for ncattr in nc_fid.variables[key].ncattrs():
                    print(f'\t\t\t{repr(nc_fid.variables[key].getncattr(ncattr))}')
            except KeyError:
                print(f'\t\t\tWARNING: does not contain variable attributes {key}')


        # NetCDF global attributes
        nc_attrs = nc_fid.ncattrs()
        if verb:
            print(f'\n\tNetCDF Global Attributes for file: {nc_file}')
            for nc_attr in nc_attrs:
                print(f'\t\t {nc_attr}, {repr(nc_fid.getncattr(nc_attr))}')

        # Dimension information.
        nc_dims = [dim for dim in nc_fid.dimensions]
        if verb:
            print(f'\n\tNetCDF dimension information for file: {nc_file}')
            for dim in nc_dims:
                print(f'\t\tName: {dim}; Size: {len(nc_fid.dimensions[dim])}')
                print_ncattr(dim)

        # Variable information.
        nc_vars = [var for var in nc_fid.variables]
        if verb:
            print(f'\n\tNetCDF variable information for file: {nc_file}')
            for var in nc_vars:
                print(
                    f'\t\tName: {var}; Dimension: {nc_fid.variables[var].dimensions}; Size: {nc_fid.variables[var].size}')
                print_ncattr(var)

        return nc_attrs, nc_dims, nc_vars


    nc_file = wnd_dir + 'uwind.nc'
    nc_fid = netCDF4.Dataset(nc_file)
    nc_attrs, nc_dims, nc_vars = ncdump(nc_file, nc_fid, verb=0)
    U_wnd = nc_fid.variables['U'][:]

    nc_file = wnd_dir + 'vwind.nc'
    nc_fid = netCDF4.Dataset(nc_file)
    V_wnd = nc_fid.variables['V'][:]

    mons = nc_fid.variables['time'][:]
    lats = nc_fid.variables['y'][:]
    lons = nc_fid.variables['x'][:]
    levs = nc_fid.variables['level'][:]

    nc_file = wnd_dir + 'Ps.nc'
    nc_fid = netCDF4.Dataset(nc_file)
    P_srf = nc_fid.variables['PS'][:]

    return U_wnd, V_wnd, P_srf, levs, lats, lons, mons


## Plot 4p

In [11]:
def fig_wind4p(map_lims):

    def one_panel(i_ax, i_var, i_uw, i_vw, i_lims, i_title, scl, i_cmap):
        vp = np.linspace(i_lims[0], i_lims[-1], 100, endpoint=True)
        i_var[i_var > i_lims[-1]] = i_lims[-1]
        my_cmap = plt.get_cmap(i_cmap)
        q = mp.quiver(x0, y0, i_uw, i_vw, i_var, angles='uv', scale_units='xy', units='xy', scale=scl, width=0.35, cmap=my_cmap)
        title = '   ' + f'{i_title}'
        if jax > -1:
            Ulb = 5*scl
            i_ax.quiverkey(q, X=0.7, Y=1.05, U=Ulb, label=str(Ulb) + 'm/s', labelpos='E')
        ax.set_title(title, loc='left', y=1.0, fontsize=f_size)

    # === interpolation
    def interp2d():

        def interp(inp_var, inp_prs, trg_prs):
            def sc_interp1d(y_prs, y_var, x_prs, method):
                func = interpolate.interp1d(y_prs, y_var, kind=method)
                x_val = func(x_prs)
                return x_val

            inp_var[np.isnan(inp_var)] = -999
            inp_prs[np.isnan(inp_prs)] = -999
            idz = np.abs(inp_prs - trg_prs).argmin()
            if any(y > trg_prs for y in inp_prs):
                out_var = sc_interp1d(inp_prs, inp_var, trg_prs, 'linear')
            else:
                out_var = inp_var[idz]
                if np.abs(trg_prs - inp_var[idz]) > trg_prs*0.1:
                    out_var = np.nan
            return out_var
        U_n = np.zeros_like(P_srf)
        V_n = np.zeros_like(P_srf)
        P_m = np.zeros_like(U_wind[0, :, :, :])

        # --- for each month
        U_m = U_wind[jm, :, :, :]
        V_m = V_wind[jm, :, :, :]
        Ps_m = P_srf[jm, :, :]

        # --- press
        for jl in range(0, len(levs)):
            P_m[jl, :, :] = Ps_m[:, :]*levs[jl]

        for ix in range(idx0, idx1):
            for iy in range(30, 50):
                U_n[jm, iy, ix] = interp(U_m[:, iy, ix], P_m[:, iy, ix], lev)
                V_n[jm, iy, ix] = interp(V_m[:, iy, ix], P_m[:, iy, ix], lev)

        return U_n[jm, :, :], V_n[jm, :, :]

    # --- domain
    idy0 = (np.abs(act_lats - map_lims[2])).argmin() - 1
    idy1 = (np.abs(act_lats - map_lims[1])).argmin() + 1
    idx0 = (np.abs(act_lons - map_lims[3])).argmin() - 1
    idx1 = (np.abs(act_lons - map_lims[4])).argmin() + 1

    # --- read nc flux
    U_wind, V_wind, P_srf, levs, lats, lons, mons = read_wind()

    # --- plot
    map_lims = map_lims[1:]
    plt.rc('font', family='serif')
    fig, axes = plt.subplots(figsize=(10, 10), nrows=2, ncols=2, sharex='all', sharey='all')
    fig.subplots_adjust(hspace=-0.2, wspace=0.1)

    f_size = 14
    scale = 1.0
    var_lims = [0, 5, 10, 15, 20, 25, 30, 35, 40]

    # --- grid
    mp = Basemap(projection='cyl', llcrnrlat=map_lims[0], urcrnrlat=map_lims[1],
                 llcrnrlon=map_lims[2], urcrnrlon=map_lims[3], resolution='l')
    x, y = np.meshgrid(lons, lats)
    x0, y0 = mp(lons, lats)

    for jax in range(0, len(axes.flat) - 0):
        jx = jax//2
        jy = jax%2
        jm = jax*3
        lev = 900
#         print(jax, jx, jy, jm, lev)

        # --- map
        ax = axes.flat[jax]
        mp.ax = ax
        mp.drawcoastlines(color='k')
        mp.drawcountries(color='k')

        UW, VW = interp2d()
        speed = np.sqrt(UW*UW + VW*VW)

        subt = ') ' + months[jm][:3] + ', ' + str(lev) + 'hPa'
        one_panel(ax, speed, UW, VW, var_lims, abc[jax] + subt, scale, 'jet')

        labp = [False, False, False, False]
        labm = [False, False, False, False]
        if jy == 0:
            labp = [True, False, True, False]
        if jx == 1:
            labm = [False, True, False, True]

        mp.drawparallels(np.arange(-90, 91, map_lims[4]), labels=labp, fontsize=f_size, color='grey')
        mp.drawmeridians(np.arange(0, 360, map_lims[5]), labels=labm, fontsize=f_size, color='grey');

    plot_name = plt_dir + '/' + 'wind_4m'
    import a_saveplot
    a_saveplot.save_plot(plot_name, ext="png", close=True, verbose=0)

In [12]:
fig_wind4p(map_lims)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


## Plot 12p

In [6]:
def fig_wind12p(map_lims):

    def one_panel(i_ax, i_var, i_uw, i_vw, i_lims, i_title, scl, i_cmap):
        vp = np.linspace(i_lims[0], i_lims[-1], 100, endpoint=True)
        i_var[i_var > i_lims[-1]] = i_lims[-1]

        my_cmap = plt.get_cmap(i_cmap)

        q = mp.quiver(x0, y0, i_uw, i_vw, i_var, angles='uv', scale_units='xy', units='xy', scale=scl, width=0.35, cmap=my_cmap)

        title = '   ' + f'{i_title}'
        if jax > -1:
            Ulb = 5*scl
            i_ax.quiverkey(q, X=0.7, Y=1.05, U=Ulb, label=str(Ulb) + 'm/s', labelpos='E')
        ax.set_title(title, loc='left', y=1.0, fontsize=f_size - 1)

    # === interpolation
    def interp2d():

        def interp(inp_var, inp_prs, trg_prs):

            def sc_interp1d(y_prs, y_var, x_prs, method):
                func = interpolate.interp1d(y_prs, y_var, kind=method)
                x_val = func(x_prs)
                return x_val


            inp_var[np.isnan(inp_var)] = -999
            inp_prs[np.isnan(inp_prs)] = -999

            idz = np.abs(inp_prs - trg_prs).argmin()
            if any(y > trg_prs for y in inp_prs):
                out_var = sc_interp1d(inp_prs, inp_var, trg_prs, 'linear')
            else:
                out_var = inp_var[idz]
                if np.abs(trg_prs - inp_var[idz]) > trg_prs*0.1:
                    out_var = np.nan
            return out_var


        U_n = np.zeros_like(P_srf)
        V_n = np.zeros_like(P_srf)
        P_m = np.zeros_like(U_wind[0, :, :, :])

        # --- for each month
        U_m = U_wind[jm, :, :, :]
        V_m = V_wind[jm, :, :, :]
        Ps_m = P_srf[jm, :, :]

        # --- press
        for jl in range(0, len(levs)):
            P_m[jl, :, :] = Ps_m[:, :]*levs[jl]

        for ix in range(idx0, idx1):
            for iy in range(30, 50):
                U_n[jm, iy, ix] = interp(U_m[:, iy, ix], P_m[:, iy, ix], lev)
                V_n[jm, iy, ix] = interp(V_m[:, iy, ix], P_m[:, iy, ix], lev)

        return U_n[jm, :, :], V_n[jm, :, :]

    # --- domain
    idy0 = (np.abs(act_lats - map_lims[2])).argmin() - 1
    idy1 = (np.abs(act_lats - map_lims[1])).argmin() + 1
    idx0 = (np.abs(act_lons - map_lims[3])).argmin() - 1
    idx1 = (np.abs(act_lons - map_lims[4])).argmin() + 1

    # --- read nc flux
    U_wind, V_wind, P_srf, levs, lats, lons, mons = read_wind()

    # --- plot
    map_lims = map_lims[1:]
    plt.rc('font', family='serif')
    fig, axes = plt.subplots(figsize=(16, 19), nrows=4, ncols=3, sharex='all', sharey='all')
    fig.subplots_adjust(hspace=-0.2, wspace=0.1)

    f_size = 13
    scale = 1.0
    var_lims = [0, 5, 10, 15, 20, 25, 30, 35, 40]

    # --- grid
    mp = Basemap(projection='cyl', llcrnrlat=map_lims[0], urcrnrlat=map_lims[1],
                 llcrnrlon=map_lims[2], urcrnrlon=map_lims[3], resolution='l')
    x, y = np.meshgrid(lons, lats)
    x0, y0 = mp(lons, lats)

    for jax in range(0, len(axes.flat) - 0):
        jx = jax//3
        jy = jax%3
        jm = jax
        lev = 900
#         print(jax, jx, jy, jm, lev)

        # --- map
        ax = axes.flat[jax]
        mp.ax = ax
        mp.drawcoastlines(color='k')
        mp.drawcountries(color='k')

        UW, VW = interp2d()
        speed = np.sqrt(UW*UW + VW*VW)

        subt = str(jy + 1) + ') ' + months[jm][:3] + ', ' + str(lev) + 'hPa'
        one_panel(ax, speed, UW, VW, var_lims, abc[jx] + subt, scale, 'jet')

        labp = [False, False, False, False]
        labm = [False, False, False, False]
        if jy == 0:
            labp = [True, False, True, False]
        if jx == 3:
            labm = [False, True, False, True]

        mp.drawparallels(np.arange(-90, 91, map_lims[4]), labels=labp, fontsize=f_size, color='grey')
        mp.drawmeridians(np.arange(0, 360, map_lims[5]), labels=labm, fontsize=f_size, color='grey');

    plot_name = plt_dir + '/' + 'wind_12m'
    import a_saveplot
    a_saveplot.save_plot(plot_name, ext="png", close=True, verbose=0)

In [5]:
fig_wind12p(map_lims)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
